In [14]:
import webvtt
from os import listdir
from os.path import isfile, join
import pandas as pd
from pydub import AudioSegment
import os


In [2]:
if(False):
    i=0
    for caption in webvtt.read('/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/Alugha_Eng/dev-clean-deu/00001/00001-1/Why_Can\'t_You_Fly%3F_deu.vtt'):
        print(caption)
        print(i)
        print('----------')
        print(caption.start) # start timestamp in text format
        print(caption.end) # end timestamp in text format
        print(caption.text) # caption text
        i=i+1

In [25]:
rootdir = "/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/Alugha_Eng/dev-clean-deu/SUB-06-19"

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        print(file)
        #print(os.path.join(subdir, file))

This_Is_Your_Brain_On_Music_DEU.wav
This_Is_Your_Brain_On_Music_DEU.vtt
AA_Battery_%2B_Gum_Wrapper_Lighter_deu.wav
AA_Battery_%2B_Gum_Wrapper_Lighter_deu.vtt
How_to_Break_Glass_with_your_Speakers_DEU.wav
How_to_Break_Glass_with_your_Speakers_DEU.vtt


In [ ]:
"""
converting all audio to wav file with the following specification. 
    -WAVE files with 16-bit
    -16 kHz
    -mono 
"""
inputdir_convert = '/home/megha/1_audio_current/audio_mp4_wav'
outputdir_convert = '/home/megha/1_audio_current/audio_mp4_wav/wave_files'
for filename in os.listdir(inputdir_convert):
    actual_filename = filename[:-4]
    if(filename.endswith(".mp4")):  #filename.endswith(".m4a") or       
        print filename
        print actual_filename
        """
        sound = AudioSegment.from_file(filename,format = "mp4")# fetch the wav file to be converted
        new_sound = sound.set_frame_rate(16000).set_channels(1)
        new_sound.export(actual_filename+".wav",format="wav")# save the wav file to the current directory. Is the file alreday exist, then it will just replace it.
        #os.system("ffmpeg -i filename -acodec pcm_s16le -ar 16000 actual_filename.wav".format(filename))
        """
        #os.system('ffmpeg -i {} -acodec pcm_s16le -ar 16000 {}.wav'.format(filename, actual_filename))
        #
        os.system('ffmpeg -i {} -acodec pcm_s16le -ac 1 -ar 16000 {}/{}.wav'.format(inputdir_convert+'/'+filename, outputdir_convert, actual_filename))
        print('--------------------------------------')
    else:
        continue

In [15]:
df  = pd.DataFrame(columns = ['A', 'B', 'C'])
for caption in webvtt.read('/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/Alugha_Eng/dev-clean-deu/00001/00001-1/Why_Can\'t_You_Fly%3F_deu.vtt'):
    df2 = pd.DataFrame([[caption.start,caption.end,caption.text]], columns=list('ABC'))
    #df['start'][i] = caption.start
    #df['end'][i] = caption.end
    #df['text'][i] = caption.text
    df = df.append(df2)
df = df.rename(index=str, columns={"A": "start", "B": "end","C" : "text"})
df
    


,start,end,text
0,00:00:00.000,00:00:02.350,Hallo und willkommen bei Life Noggin. Wie cool...
0,00:00:02.351,00:00:05.388,wenn wir fliegen könnten? Leider ist das zu sc...
0,00:00:05.389,00:00:08.600,Aber warum? Um das zu beantworten müssen wir d...
0,00:00:08.601,00:00:11.377,der Physik verstehen. Sir Isaac Newtons zweites
0,00:00:11.378,00:00:18.158,"Gesetz der Bewegung besagt, dass die Summe der..."
0,00:00:18.159,00:00:22.676,Körper anstoße wirkt eine resultierende Kraft ...
0,00:00:22.677,00:00:29.022,alle Kräfte die auf einen Körper wirken sich a...
0,00:00:29.023,00:00:33.279,gar nicht bewegen. Und genau das passiert wenn...
0,00:00:33.280,00:00:37.275,"Die Schwerkraft, die es nach unten drückt wird..."
0,00:00:37.276,00:00:42.576,"Wenn ein Flugzeug fliegt, sind seine Tragfläch..."


In [ ]:
os.system('ffmpeg -i {} -acodec pcm_s16le -ac 1 -ar 16000 {}/{}.wav'.format(inputdir_convert+'/'+filename, outputdir_convert, actual_filename))

In [21]:
SAVE_DIR = "/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/Alugha_Eng/dev-clean-deu/"
dirName = "Why Can't You Fly_mixdown_ln_cantFly_DEU" # direct file name as directory name
dirPath = SAVE_DIR+"/"+dirName
#here you put a loop to read all the wav files
myaudio = AudioSegment.from_file("/media/home/megha/5_wav2letter/WAV_2_LETTER/wav2letter/tutorials/Alugha_Eng/dev-clean-deu/00001/00001-1/Why Can't You Fly_mixdown_ln_cantFly_DEU.wav", "wav")
for i in range(0, len(df)):
    if not os.path.exists(dirPath):
        os.makedirs(dirPath)    
    start_time = df['start'].iloc[i]
    end_time = df['end'].iloc[i]    
    # ------------------------------------------------------------------------
    s_hours, s_minutes, s_seconds = (["0", "0"] + start_time.split(":"))[-3:]
    s_hours = int(s_hours)
    s_minutes = int(s_minutes)
    s_seconds = float(s_seconds)
    s_millisecond = int(3600000 * s_hours + 60000 * s_minutes + 1000 * s_seconds)
    START = s_millisecond
    #--------------------------------------------------------------------------
    e_hours, e_minutes, e_seconds = (["0", "0"] + end_time.split(":"))[-3:]
    e_hours = int(e_hours)
    e_minutes = int(e_minutes)
    e_seconds = float(e_seconds)
    e_miliseconds = int(3600000 * e_hours + 60000 * e_minutes + 1000 * e_seconds)
    END = e_miliseconds
    #--------------------------------------------------------------------------
    chunk = myaudio[START:END]    
    chunk_name =  str(i)+".wav" 
    file_destination = dirPath+"/"+chunk_name
    if not os.path.exists(file_destination):
        print("exporting", chunk_name)
        chunk.export(file_destination , format="wav")



exporting 0.wav
exporting 1.wav
exporting 2.wav
exporting 3.wav
exporting 4.wav
exporting 5.wav
exporting 6.wav
exporting 7.wav
exporting 8.wav
exporting 9.wav
exporting 10.wav
exporting 11.wav
exporting 12.wav
exporting 13.wav
exporting 14.wav
exporting 15.wav
exporting 16.wav
exporting 17.wav
exporting 18.wav
exporting 19.wav
exporting 20.wav
exporting 21.wav
exporting 22.wav
exporting 23.wav
